In [ ]:
!pip install --upgrade --no-cache-dir gdown
!pip install diffusers
!pip install datasets
!pip install peft
!pip install gradio

In [ ]:
import gdown
url = "https://drive.google.com/uc?id=1mR_axgYLVaFzLJ3QZw8NZlTwk6WTpcXW"
output = "/content/code.zip"
gdown.download(url, output, quiet=False)

In [ ]:
import zipfile
import os

zip_file_path = '/content/code.zip'
extract_folder_path = '/content/'

# Check if the zip file exists
if os.path.exists(zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_folder_path)
else:
    print("The specified zip file does not exist.")


In [ ]:
import torch
device = 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

In [ ]:
from models import Diffusion
d = Diffusion(device , "LMSD")

In [ ]:
!pip install -U git+https://github.com/openai/CLIP.git

In [ ]:
import torch
import clip
from PIL import Image
model, preprocess = clip.load('ViT-B/32')
model = model.to(device)
def get_clip_score(image, text):
# Load the pre-trained CLIP model and the image

    # Preprocess the image and tokenize the text
    image_input = preprocess(image).unsqueeze(0)
    text_input = clip.tokenize([text] , truncate=True)

    # Move the inputs to GPU if available
    device = "cuda" if torch.cuda.is_available() else "cpu"
    image_input = image_input.to(device)
    text_input = text_input.to(device)


    # Generate embeddings for the image and text
    with torch.no_grad():
        image_features = model.encode_image(image_input)
        text_features = model.encode_text(text_input)

    # Normalize the features
    image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)

    # Calculate the cosine similarity to get the CLIP score
    clip_score = torch.matmul(image_features, text_features.T).item()

    return clip_score


100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 128MiB/s]


In [ ]:
from datasets import load_dataset

In [ ]:
evaluate_dataset = load_dataset("nateraw/parti-prompts", split="train")

In [ ]:
arrScores = []

In [ ]:
for i in range(int(len(evaluate_dataset)//5)):
  Prompts = []
  for j in range(5):
    Prompts.append(evaluate_dataset[i*5 + j]["Prompt"])
  ims,_ = d.generate(Prompts , scale_guide=11, timesteps = 50 ,dataset_ = "None", resolution=(512,512))
  for j in range(5):
    score = get_clip_score(ims[j], Prompts[j])
    sample = evaluate_dataset[i*5 + j]
    s = {"Prompt" : sample["Prompt"] ,"image" : ims[j], "Category" :sample["Category"] , "Challenge" :sample["Challenge"] , "CLIP Score":score }
    arrScores.append(s)
    print(f"{s}")


In [ ]:
categories ={}
challenges ={}

In [ ]:
for i in range(len(arrScores)):
  if (arrScores[i]["Category"] not in categories):
    categories[arrScores[i]["Category"]]=0
  if (arrScores[i]["Challenge"] not in challenges):
    challenges[arrScores[i]["Challenge"]]=0

In [ ]:
for key in categories:
  sum_ = 0
  len_ = 0
  for i in range(len(arrScores)):
    if (arrScores[i]["Category"] == key):
      sum_ += arrScores[i]["CLIP Score"]
      len_+=1
  categories[key] = sum_/len_

In [ ]:
for key in challenges:
  sum_ = 0
  len_ = 0
  for i in range(len(arrScores)):
    if (arrScores[i]["Challenge"] == key):
      sum_ += arrScores[i]["CLIP Score"]
      len_+=1
  challenges[key] = sum_/len_

In [ ]:
sum_ = 0
len_ = 0
for i in range(len(arrScores)):
    sum_ += arrScores[i]["CLIP Score"]
    len_+=1
sum_/len_